In [ ]:
from collections import Counter
sorted([(name, Counter(name[0].lower().translate(dict.fromkeys(map(ord, 'aeiounrst .-'), None)))) for name in Card.objects.values_list('name')], key=lambda x: max(x[1].values() or [0]), reverse=True)


In [ ]:
from collections import Counter
Counter(''.join([name[0].lower() for name in Card.objects.values_list('name')]))

In [ ]:
from cards.models import *
print(f'Cards: {Card.objects.count()}')

In [ ]:
print(f'CardPrintings: {CardPrinting.objects.count()}')

In [ ]:
print(f'CardPrintingLanguages: {CardPrintingLanguage.objects.count()}')

In [ ]:
print(f'PhysicalCards: {PhysicalCard.objects.count()}')

In [ ]:
print(f'UserOwnedCards: {UserOwnedCard.objects.filter(owner=User.objects.get(username="Liam")).count()}')

In [ ]:
from django.db.models import Sum
print(f"My Cards: {UserOwnedCard.objects.filter(owner=User.objects.get(username='Liam')).aggregate(Sum('count'))}")

In [ ]:
from django.db import connection
with connection.cursor() as cursor:
    cursor.execute("""
SELECT SUM(reltuples) rowcount
FROM pg_class C
LEFT JOIN pg_namespace N ON (N.oid = C.relnamespace)
WHERE nspname NOT IN ('pg_catalog', 'information_schema') 
AND relkind='r' 
""")
    result = cursor.fetchone()
    
print(result)

In [3]:
for card in Card.objects.filter(is_token=True).all():
    card.name = f"{card.name} ({card.scryfall_oracle_id.split('-')[0]})"
    card.save()

In [6]:
Card.objects.filter(id__in=[29142, 29143, 29145, 29146, 29147, 29148, 29149, 29151, 29152, 29154, 29155, 29156, 29157, 29158, 29159, 29170, 29117, 29118, 29131, 29132, 29141, 29144, 29150, 29153, 29160, 29169]).delete()

(130,
 {'cards.Card_links': 0,
  'cards.DeckCard': 0,
  'cards.CardPrintingLanguage_physical_cards': 26,
  'cards.CardImage': 26,
  'cards.CardRuling': 0,
  'cards.CardLegality': 0,
  'cards.CardTag_cards': 0,
  'cards.CardPrintingLanguage': 26,
  'cards.CardPrinting': 26,
  'cards.Card': 26})